# Friends_DialoGPT 모델링

In [1]:
!git clone https://github.com/TMTsProject/django-chatbot.git
%cd django-chatbot/

Cloning into 'django-chatbot'...
remote: Enumerating objects: 609, done.
remote: Counting objects: 100% (609/609), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 609 (delta 280), reused 495 (delta 191), pack-reused 0
Receiving objects: 100% (609/609), 30.38 MiB | 22.33 MiB/s, done.
Resolving deltas: 100% (280/280), done.
/content/django-chatbot


In [2]:
! pip -q install transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 895 kB 38.5 MB/s 
     |████████████████████████████████| 3.3 MB 34.3 MB/s 
     |████████████████████████████████| 596 kB 42.9 MB/s 


In [3]:
#import 
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


#데이터셋

preprocessing을 통해 생성한 Friends.csv파일을 사용합니다.

In [4]:
data = pd.read_csv('Data/friends_raw_data/Friends.csv')

FileNotFoundError: ignored

In [ ]:
contexted = []
n = 3
for i in range(n, len(data['Line'])):
    if data['Character'][i] == 'rachel':
        row = []
        prev = i - 1 - n
        if data['Scene'][i-n] == data['Scene'][i]:
            for j in range(i, prev, -1):
                row.append(data['Line'][j])
            contexted.append(row)  
    
columns = ['response', 'context']
columns = columns + ['context_'+str(i+2) for i in range(n-1)]
df = pd.DataFrame.from_records(contexted, columns=columns)

In [ ]:
data.head()

,Scene,Character,Line
0,1,monica,There's nothing to tell! He's just some guy I work with!
1,1,joey,"C'mon, you're going out with the guy! There's gotta be something wrong with him!"
2,1,chandler,"All right Joey, be nice. So does he have a hump? A hump and a hairpiece?"
3,1,phoebe,"Wait, does he eat chalk?"
4,1,phoebe,"Just, 'cause, I don't want her to go through what I went through with Carl- oh!"


데이터 프레임 크기
- rachel : 8027
- joey : 6974
- ...

Rachel의 발화 수가 가장 많아, Rachel의 페르소나를 학습한 모델을 제작하였습니다.

In [ ]:
df.head()

,response,context,context_2,context_3
0,Oh God Monica hi! Thank God! I just went to yo...,Rachel?!,And I just want a million dollars!,"I don't want to be single, okay? I just... I j..."
1,"Hi, sure!","This is everybody, this is Chandler, and Phoeb...",Can I get you some coffee?,Oh God Monica hi! Thank God! I just went to yo...
2,"Oh God... well, it started about a half hour b...","So you wanna tell us now, or are we waiting fo...",Hi.,"Hi, sure!"
3,"Ooh, I was kinda hoping that wouldn't be an is...",Who wasn't invited to the wedding.,"Oh God... well, it started about a half hour b...","So you wanna tell us now, or are we waiting fo..."
4,"Daddy, I just... I can't marry him! I'm sorry....",I'll have whatever Christine is having.,Tuna or egg salad? Decide!,Now I'm guessing that he bought her the big pi...


In [ ]:
# train과 test 사이즈는 8:2의 비율로 제작하였습니다
trn_df, val_df = train_test_split(df, test_size = 0.2)
trn_df.head()

,response,context,context_2,context_3
1748,"Oh honey, come on, I’m sorry, I didn’t.... I don’t mind paying my dues, y'know, its just how much am I gonna learn about fashion by walking Mira, the arthritic seamstress, to the bathroom.","Gee, I don’t know Rach. Order up!! I got a Yentel soup, a James Beans, and a Howdy hold the Dowdy!","Oh, I wanna quit, but then I think I should stick it out, then I think why would such a person stay in such a demeaning job, just because it’s remotely related to the field they’re interested in.","I know honey, I’m sorry."
4088,"Yes, but still. They're people…with eyes.",But people that you don't know and will never see again.,Because they're people.,Why do you care?
3317,"That's right! 'Cause that would give him the control! So now he's all ooh, coming up with this whole I've got a party thing y'know, trying to get me to hint around for an invitation. Blew up in his face, didn't it?",And you're not gonna do that.,"Okay. All right, I see what he's doing! He's not asking me out, because he wants me to ask him out.","Well, I guess we won't be warming his house."
4270,Jill! Did you shop?!,Hey!,Hey! What’s goin’ on?,I would love that job!
111,"Look at you, you are so big I can't believe it!",Look at you in the apron. You look like you're in a play.,"Well, we were in the city shopping, and your mom said you work here, aaand it's true!",What are you guys doing here?


## DialoGPT 모델링

In [ ]:
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""


# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = 'output-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 5
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [ ]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
# Main runner

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelForCausalLM.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

08/21/2021 08:09:38 - INFO - filelock -   Lock 139760705367504 acquired on /root/.cache/huggingface/transformers/5f8cf488e0bdda2e393e798f478673a4d26c1386082a1a05e42269f3ecc89f50.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

08/21/2021 08:09:38 - INFO - filelock -   Lock 139760705367504 released on /root/.cache/huggingface/transformers/5f8cf488e0bdda2e393e798f478673a4d26c1386082a1a05e42269f3ecc89f50.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock
08/21/2021 08:09:39 - INFO - filelock -   Lock 139760700715536 acquired on /root/.cache/huggingface/transformers/3cf340c89a43b5e6f31c4cd609fc2fc92f3d7aafdf6c8987e2ea9e02cb78b4e2.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

08/21/2021 08:09:40 - INFO - filelock -   Lock 139760700715536 released on /root/.cache/huggingface/transformers/3cf340c89a43b5e6f31c4cd609fc2fc92f3d7aafdf6c8987e2ea9e02cb78b4e2.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock
08/21/2021 08:09:41 - INFO - filelock -   Lock 139760701336400 acquired on /root/.cache/huggingface/transformers/4e3f74e7c741909c4d1b48a23febe75c1be66a20c2b98cf7db4b8b10f12dc10c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

08/21/2021 08:09:41 - INFO - filelock -   Lock 139760701336400 released on /root/.cache/huggingface/transformers/4e3f74e7c741909c4d1b48a23febe75c1be66a20c2b98cf7db4b8b10f12dc10c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
08/21/2021 08:09:44 - INFO - filelock -   Lock 139760354540880 acquired on /root/.cache/huggingface/transformers/aeb12aa1fc2f135700fcf9f8f0eec86c0649dc5ce0df86677adf0388271f33f3.1010e0ba25016a38144b58e8852f1dcc18876341e3b5728a99b3ffa11cc733cd.lock


Downloading:   0%|          | 0.00/351M [00:00<?, ?B/s]

08/21/2021 08:09:57 - INFO - filelock -   Lock 139760354540880 released on /root/.cache/huggingface/transformers/aeb12aa1fc2f135700fcf9f8f0eec86c0649dc5ce0df86677adf0388271f33f3.1010e0ba25016a38144b58e8852f1dcc18876341e3b5728a99b3ffa11cc733cd.lock


In [ ]:
main(trn_df, val_df)

08/21/2021 08:11:25 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/21/2021 08:11:42 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7f1c92ef1bd0>
08/21/2021 08:11:42 - INFO - __main__ -   Creating features from dataset file at cached
08/21/2021 08:11:48 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
08/21/2021 08:11:48 - INFO - __main__ -   ***** Running training *****
08/21/2021 08:11:48 - INFO - __main__ -     Num examples = 6421
08/21/2021 08:11:48 - INFO - __main__ -     Num Epochs = 5
08/21/2021 08:11:48 - INFO - __main__ -     Instantaneous batch size per GPU = 4
08/21/2021 08:11:48 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 4
08/21/2021 08:11:48 - INFO - __main__ -     Gradient Accumulation steps = 1
08/21/2021 08:11:48 - INFO - __main__ -     Total optimization steps = 8025


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1605 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Iteration:   0%|          | 0/1605 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1605 [00:00<?, ?it/s]

08/21/2021 08:48:36 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-7000
08/21/2021 08:49:10 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-7000
08/21/2021 08:54:28 - INFO - __main__ -    global_step = 8025, average loss = 1.7615785457337756
08/21/2021 08:54:28 - INFO - __main__ -   Saving model checkpoint to output-small
08/21/2021 08:54:42 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small']
08/21/2021 08:54:44 - INFO - __main__ -   Creating features from dataset file at cached
08/21/2021 08:54:47 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
08/21/2021 08:54:47 - INFO - __main__ -   ***** Running evaluation  *****
08/21/2021 08:54:47 - INFO - __main__ -     Num examples = 1606
08/21/2021 08:54:47 - INFO - __main__ -     Batch size = 4


Evaluating:   0%|          | 0/401 [00:00<?, ?it/s]

08/21/2021 08:55:19 - INFO - __main__ -   ***** Eval results  *****
08/21/2021 08:55:19 - INFO - __main__ -     perplexity = tensor(5.9890)


{'perplexity_': tensor(5.9890)}

In [ ]:
import torch
model.save_pretrained("static/friends3_epoch5_small_pretrained")
tokenizer.save_pretrained("static/friends3_epoch5_small_tokenizer")

('model/friends3_epoch5_small_tokenizer/tokenizer_config.json',
 'model/friends3_epoch5_small_tokenizer/special_tokens_map.json',
 'model/friends3_epoch5_small_tokenizer/vocab.json',
 'model/friends3_epoch5_small_tokenizer/merges.txt',
 'model/friends3_epoch5_small_tokenizer/added_tokens.json',
 'model/friends3_epoch5_small_tokenizer/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
MODEL_PATH='static/'
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH + "friends3_epoch5_small_pretrained")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH + "friends3_epoch5_small_tokenizer")

## 문법 교정 API 연동

In [ ]:
import requests
from urllib.parse import unquote, quote, quote_plus, urlencode

# encoding
def encodeInput(text):
    encoded_text = "text=" + quote(text) + "&language=en-US"
    return encoded_text
    
# language tool api
def languageTool(text):
    url = "https://dnaber-languagetool.p.rapidapi.com/v2/check"
    
    inputText = encodeInput(text)
    headers = {
        'content-type': "application/x-www-form-urlencoded",
        'x-rapidapi-key': "49491d0e58mshc6937e8d420377ap1de6f6jsn6ea9c2dcaa0b",
        'x-rapidapi-host': "dnaber-languagetool.p.rapidapi.com"
    }

    response = requests.request("POST", url, data=inputText, headers=headers)
    matches =  response.json()['matches']
    
    if matches == []:
       # 맞는 문장
        #msg = "Correct!"
        msg = ""
    else:
        #틀린 문장    
        msg = ""
				# 오류가 여러개면 여러개 출력하도록 함.
        for m in range(len(matches)):
            err =  matches[m]['shortMessage']
            # spelling mistake : 철자오류 또는 띄어쓰기 오류
            # 그런 종류의 오류는 무시함
            if err == 'Spelling mistake' :
                msg += "There is a spelling mistake. The chatbot may not work smoothly.\n"
                continue
            elif err == '':
                continue
            msg += err + " : " + matches[m]['message'] + '\n'
            msg += 'Can be replaced by : '
            rep = matches[m]['replacements']
            for i in range(len(rep)):
                msg += rep[i]['value']
                if i != len(rep)-1:
                    msg += ', '
            msg += '\n'
    return msg

In [ ]:
def subIdx(a):
    v = a.pop(0)
    for i in range(len(a)):
        a[i] -= v
    return v

In [ ]:
new_user_input_ids = None
bot_input_ids = None
chat_history_ids = None

step = 0
chat_history_idx = []

while True:
    user_input_text = input(">> User: ")
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(user_input_text + tokenizer.eos_token, return_tensors='pt')
    
    ## grammar bot
    print(languageTool(user_input_text), end='')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids != None else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens
    #샘플링 방식임
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature = 0.8
    )
    step += 1
    
    # pretty print last ouput tokens from bot
    print("Rachel: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    chat_history_idx.append(chat_history_ids.shape[-1])

    #오래된 히스토리는 삭제하여 대화가 원활하게 이어지도록 함.
    if step > 5:
        cv = subIdx(chat_history_idx)
        chat_history_ids = chat_history_ids[:, cv:]


>> User: Hello, Joey
Joey: Hi, Dan
>> User: Can you tell me about Dr. Drake Ramore?
Joey: Dr. Drake Ramor
>> User: Yes. Can you tell me about him?
Joey: I know of him, he is a good doctor.
>> User: Oh, thanks.
Joey: It's fine.
>> User: What is your favorite food?
Joey: Mm. Chicken.
>> User: How about sandwich and beer?
Joey: Yeah. I like that.
>> User: What do you do?
Joey: Well, I am a professor of the city of Alexandria.
>> User: Are you professor?
Joey: Yes. He's also a teacher.
>> User: Who are you?
Joey: A person who knows a lot about a lot of things.
>> User: bye
Joey: Bye. Bye.
>> User: Nice to meet you
Joey: You too.
>> User: Do you like fries?
Joey: I do!
>> User: Can I eat your fries?
Joey: You can!
>> User: Joey doesn't share food. Right?
Joey: Joeys the best!
>> User: I don't think so
Joey: Yeah, he is.
>> User: Umm
Joey: What's your name?
>> User: I'm Rachel.
Joey: Rachel's a good name.
>> User: Thanks. What's your name?
Joey: I have a name. It's not Rachel though.
>> User

KeyboardInterrupt: ignored